In [1]:
# pip install amazon-textract-response-parser

In [5]:
import boto3
from IPython.display import Image, display
from trp import Document
from PIL import Image as PImage, ImageDraw
import time
from IPython.display import IFrame
import sagemaker
import json
from tqdm import tqdm
import pickle

In [6]:
mySession = boto3.session.Session()
awsRegion = mySession.region_name
s3BucketName = "ocr-nlp-vsu"
account_id = boto3.client('sts').get_caller_identity().get('Account')
role = sagemaker.get_execution_role()


In [7]:
# Amazon S3 client
s3 = boto3.client('s3')

# Amazon Textract client
textract = boto3.client('textract')

In [8]:
def get_file_list(s3_client,s3BucketName, Prefix=None):
    
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket='ocr-nlp-vsu', Prefix=Prefix)
    response = s3_client.list_objects_v2(Bucket=s3BucketName)
    file_list = []
    for page in pages:
        for obj in page['Contents']:
            file_list.append(obj['Key']
                            )
    return file_list    

In [9]:
def extract_fields(s3BucketName,documentName,s3_client,textract_client,out='json',err_out=False):
    # Call Amazon Textract
    captured_errors = []
    error_list = []

    try:
    
        response = textract_client.analyze_document(
            Document={
                'S3Object': {
                    'Bucket': s3BucketName,
                    'Name': documentName
                }
            },
            FeatureTypes=["FORMS"])

    except Exception as e:
        time.sleep(30)
        error_list.append(e)
        captured_errors.append(documentName)
        
        
    #collect response

    doc = Document(response)

    dict_ext={}
    for page in doc.pages:
        print("Start Fields Extraction")
        for field in page.form.fields:
            if hasattr(field.value, "confidence"): 
                field_confidence = field.value.confidence
            else:
                field_confidence = float('nan')
        
            key_field_couple = {str(field.key): {'value':str(field.value),'confidence':field_confidence}}
            dict_ext.update(key_field_couple)
    print("End Fields Extraction")

    if out=='dict':
        if err_out:
            return dict_ext, response, error_list, captured_errors
        else:
            return dict_ext, response
    
    else:
        if err_out:
            return json.dumps(dict_ext), response, error_list, captured_errors 
        else:
            return json.dumps(dict_ext), response

In [13]:
def docs_extract_fields(documentName_list,err_out=False):
    
    doc_dict_ext = []
    res_dict = {}
    for document in tqdm(documentName_list):
        
        parsed_document = document.split('/')[-1]
        print("Processing Document: {}".format(parsed_document))
        if err_out:
            dict_ext, response, _error_list, _captured_errors = extract_fields(
                s3BucketName,document,s3,textract,out='dict',err_out=True)
        else:
            dict_ext, response = extract_fields(s3BucketName,document,s3,textract,out='dict')
        
        dict_ext.update({'image_name':parsed_document})
        doc_dict_ext.append(dict_ext)
        
        res_dict[parsed_document] = response
       
    print('End of Document Processing')

    if err_out:
        return doc_dict_ext, res_dict, _error_list, _captured_errors
    else:
        return doc_dict_ext, res_dict 

In [11]:
documentName_list = get_file_list(s3,s3BucketName, Prefix='nist_data/')
len(documentName_list)

2100

## Extract fields from all documents

In [10]:
dict_out, resp_dict_out = docs_extract_fields(documentName_list)

  0%|          | 0/2100 [00:00<?, ?it/s]

Processing Document: f0000_14.png


  0%|          | 1/2100 [00:00<30:03,  1.16it/s]

Start Fields Extraction
End Fields Extraction
Processing Document: f0001_41.png


  0%|          | 2/2100 [00:01<30:28,  1.15it/s]

...

Start Fields Extraction
End Fields Extraction
Processing Document: f2098_02.png


100%|█████████▉| 2099/2100 [36:41<00:01,  1.01s/it]

Start Fields Extraction
End Fields Extraction
Processing Document: f2099_21.png


100%|██████████| 2100/2100 [36:41<00:00,  1.05s/it]

Start Fields Extraction
End Fields Extraction
End of Document Processing


## Serialise objects and write files into S3 bucket

In [42]:
pickled_textract_response_dict = pickle.dumps(resp_dict_out)
pickled_textract_dictionary = pickle.dumps(dict_out)

In [43]:
s3.put_object(Body=pickled_textract_response_dict, Bucket=s3BucketName, 
                  Key='textract_dict_response.pkl')
s3.put_object(Body=pickled_textract_dictionary, Bucket=s3BucketName, 
                  Key='textract_dictionary.pkl')

{'ResponseMetadata': {'RequestId': '6Q9NZQ2S53P143GX',
  'HostId': 'wOtejJuC+YTqoRwcX3mrj/H+/uR2AjByd39rOaCkw+jrS+vybXgVxLRtu1rPkDdKBDH3vDUGiirElGtlOyxs4w==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wOtejJuC+YTqoRwcX3mrj/H+/uR2AjByd39rOaCkw+jrS+vybXgVxLRtu1rPkDdKBDH3vDUGiirElGtlOyxs4w==',
   'x-amz-request-id': '6Q9NZQ2S53P143GX',
   'date': 'Mon, 17 Jul 2023 16:26:30 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"473b6860022eb92c1f91b3c84027e810"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"473b6860022eb92c1f91b3c84027e810"',
 'ServerSideEncryption': 'AES256'}

## Read and load files
### Responses

In [ ]:
txtrct_dict_response = s3.get_object(Bucket=s3BucketName, Key='textract_dict_response.pkl')

In [19]:
# txtrct_dict_response

In [17]:
txtrct_dict_response_body = txtrct_dict_response['Body'].read()
txtrct_dict_response_data = pickle.loads(txtrct_dict_response_body)

In [20]:
# txtrct_dict_response_data

### Dictionary

In [ ]:
txtrct_dict = s3.get_object(Bucket=s3BucketName, Key='textract_dictionary.pkl')

In [21]:
# txtrct_dict

In [ ]:
txtrct_dict_body = txtrct_dict['Body'].read()
txtrct_dict_data = pickle.loads(txtrct_dict_body)

In [22]:
# txtrct_dict_data